In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [4]:
books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [5]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [6]:
len(df_ratings)

1149780

In [7]:
# Group by 'user' and count the number of ratings for each user
user_rating_counts = df_ratings.groupby('user').size().reset_index(name='rating_count')

# Filter users who have less than 100 ratings
filtered_users = user_rating_counts[user_rating_counts['rating_count']> 200].drop('rating_count', axis=1)

# Display the users with less than 100 ratings
print(filtered_users)

          user
95         254
791       2276
981       2766
1049      2977
1177      3363
...        ...
104147  275970
104688  277427
104706  277478
104768  277639
105111  278418

[899 rows x 1 columns]


In [8]:
# Group by 'isbn' and count the number of ratings for each book
book_rating_counts = df_ratings.groupby('isbn').size().reset_index(name='rating_count')

# Filter books with than 200 ratings
filtered_books = book_rating_counts[book_rating_counts['rating_count']> 100].drop('rating_count', axis=1)

filtered_books

,isbn
3800,002542730X
5416,0060008032
5683,0060096195
6204,006016848X
6338,0060173289
...,...
262739,1573227331
262803,1573229326
262813,1573229571
268744,1592400876


In [9]:
filtered_df_ratings = df_ratings[
    df_ratings['user'].isin(filtered_users['user']) &
    df_ratings['isbn'].isin(filtered_books['isbn'])
    ]

In [10]:
filtered_df_ratings = pd.merge(filtered_df_ratings, df_books, on='isbn', how='inner')
filtered_df_ratings.drop(['isbn', 'author'], axis=1, inplace=True)

In [11]:
len(filtered_df_ratings)

48990

In [12]:
filtered_df_ratings.dtypes

user        int32
rating    float32
title      object
dtype: object

In [13]:
filtered_df_ratings.drop_duplicates(subset=['user', 'title'], inplace=True)

In [14]:
user_item_matrix = filtered_df_ratings.pivot(index='title', columns='user', values='rating').fillna(0)
user_item_matrix

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Assuming `user_item_matrix` is a DataFrame, convert it to csr_matrix
user_item_matrix_csr = csr_matrix(user_item_matrix.values)
print(type(user_item_matrix_csr))  # Check to confirm it's now a csr_matrix

<class 'scipy.sparse._csr.csr_matrix'>


In [16]:
# add your code here - consider creating a new cell for each section of code
model = NearestNeighbors(n_neighbors=5, metric='cosine')

In [19]:
model.fit(user_item_matrix_csr)

,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [20]:
user_item_matrix_csr.shape  # Output should be (number_of_users, number_of_books)

(664, 882)

In [21]:
print("Number of neighbors:", model.n_neighbors)
print("Metric used:", model.effective_metric_)

Number of neighbors: 5
Metric used: cosine


In [22]:
row_index = 654  # Use an appropriate row index based on your matrix size
row = user_item_matrix_csr.getrow(row_index)  # This returns a sparse matrix with only the selected row
dense_row = row.toarray().reshape(1,-1)
distances, indices = model.kneighbors(dense_row, n_neighbors=5)

In [23]:
print("Distances to nearest neighbors:", distances)
print("Indices of nearest neighbors:", indices)

Distances to nearest neighbors: [[0.         0.69087744 0.749305   0.7583527  0.759055  ]]
Indices of nearest neighbors: [[654 116 167 505  48]]


In [24]:
# Assuming indices is a 2D array and we want the first row for nearest neighbors
nearest_indices = indices.flatten()  # Flatten to convert to 1D array

# Get the titles corresponding to these indices
nearest_titles = user_item_matrix.index[nearest_indices].tolist()  # Access titles using the indices

print("Nearest Titles:", nearest_titles)

Nearest Titles: ['Wicked: The Life and Times of the Wicked Witch of the West', 'Confessions of an Ugly Stepsister : A Novel', 'Empire Falls', 'The Honk and Holler Opening Soon', 'Angus, Thongs and Full-Frontal Snogging: Confessions of Georgia Nicolson']


In [25]:
book_title = 'The Queen of the Damned (Vampire Chronicles (Paperback))'
try:
    # Get the row index for the given book title
    row_index = user_item_matrix.index.get_loc(book_title)
    print(row_index)
except KeyError:
    print(f"Book title '{book_title}' not found in the user-item matrix.")
    

559


In [26]:
import math

# function to return recommended books - this will be tested
def get_recommends(book_title = ""):
    try:
    # Get the row index for the given book title
        row_index = user_item_matrix.index.get_loc(book_title)
        row = user_item_matrix_csr.getrow(row_index)  # This returns a sparse matrix with only the selected row
        dense_row = row.toarray()
        distances, indices = model.kneighbors(dense_row, n_neighbors=6)
        distances = distances.tolist()
        # Assuming indices is a 2D array and we want the first row for nearest neighbors
        nearest_indices = indices.flatten()  # Flatten to convert to 1D array
        # Get the titles corresponding to these indices
        nearest_titles = user_item_matrix.index[nearest_indices].tolist()  # Access titles using the indices
        result = []
        for i in range(1,6):
            item = [nearest_titles[i], round(distances[0][i], 2)]
            result.insert(0, item)
        return [book_title, result]
    except KeyError:
        print(f"Book title '{book_title}' not found in the user-item matrix.")
        return None

In [27]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
for i in range(5):
    print(books[1][i])

["I'll Be Seeing You", 0.8]
['The Weight of Water', 0.77]
['The Surgeon', 0.77]
['I Know This Much Is True', 0.77]
['The Lovely Bones: A Novel', 0.72]


In [28]:
def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      print(f"{i}th book not in")
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

You passed the challenge! 🎉🎉🎉🎉🎉
